# タスク 3: 質問応答に Amazon Bedrock を使用する

このノートブックでは、関連する完全なコンテキストを含むリクエストをモデルに送信し、応答が返されるのを待つことで、クエリに対する情報の応答をAmazon Bedrockを通じて Nova Lite モデルから得る方法を学習します。これにより、事前にドキュメントを準備してインデックスを作成することなく、モデルが質問に対して事実に基づいた応答を返すという課題に対処します。

このノートブックは、**Retrieval-Augmented Generation (RAG)** が行うことをシミュレートしますが、実際には RAG を使用しません。このアプローチは、短いドキュメントまたはシングルトンアプリケーションで機能します。モデルに送信されるプロンプトに収まらないような大規模なエンタープライズドキュメントを使用するエンタープライズレベルの質問応答には拡張できない可能性があります。

**質問応答 (QA)** は、自然言語で提示された事実に基づくクエリに対する応答を抽出する重要なタスクです。通常、QA システムは、構造化データまたは非構造化データを含むナレッジ ベースに対してクエリを処理し、正確な情報を含む応答を生成します。高い精度を確保することは、特にエンタープライズ ユース ケースにおいて、有用で信頼性が高く、信頼できる質問応答システムを開発する上で重要です。


## シナリオ

AnyCompany で、同社が製造する特定の車両モデルのタイヤ交換に関する情報を提供するよう質問応答モデルに求める状況をモデル化してみます。まず、「ゼロショット」アプローチを使用してモデルにクエリを実行し、トレーニングデータのみに基づいて適切な回答を提供できるかどうかを確認します。

ただし、偽の車両モデルを試して同様の応答が得られた場合、モデルがより一般的な回答を「幻覚」しているように見えることがわかります。これは、各モデルのタイヤの詳細を提供するために、Example Company の実際の車両マニュアルを使用してモデルのトレーニングを拡張する必要があることを意味します。

このラボでは、外部データなしでこのような「検索拡張生成」(RAG) アプローチをシミュレートします。AnyCompany Model Z 車両のタイヤ交換方法を説明した詳細なマニュアルの抜粋を提供します。このコンテキスト内のサンプル コンテンツを活用して、モデルがカスタマイズされた正確な回答を提供できるかどうかをテストします。

## Task 3.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



In [ ]:
"""
Nova Lite Adapter:

This code block contains helper functions for using Nova Lite.
"""

import json
import time
from botocore.exceptions import ClientError

def format_for_nova_lite(prompt_text):
    """Format the prompt for Nova Lite's expected message structure."""
    return {
        "messages": [
            {
                "role": "user",
                "content": [{"text": prompt_text}]
            }
        ],
        "inferenceConfig": {
            "maxTokens": 2048,
            "temperature": 0,
            "topP": 0.9
        }
    }

def parse_nova_lite_response(response_body):
    """Parse the response from Nova Lite."""
    if 'output' in response_body and 'message' in response_body['output']:
        message = response_body['output']['message']
        if 'content' in message and isinstance(message['content'], list):
            # Extract text from each content item
            texts = []
            for content_item in message['content']:
                if isinstance(content_item, dict) and 'text' in content_item:
                    texts.append(content_item['text'])
            return ' '.join(texts)
    
    # Fallback if the response format is different
    return str(response_body)

## タスク 3.2: モデルの知識を使用した Q&A
このセクションでは、Bedrock サービスによって提供されるモデルを使用して、トレーニングフェーズ中に獲得した知識に基づいて質問に答えます。

このタスクでは、Amazon Bedrock クライアントのinvoke_model() メソッドを使用します。このメソッドを使用するために必要な必須パラメータは、Amazon Bedrock モデル ARN を表す modelId と、タスクのプロンプトである body です。

body プロンプトは、選択した基盤モデル プロバイダーに応じて変わります。これについては、以下で詳しく説明します。

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Bedrock サービスによって提供されるモデルを使用して、トレーニング フェーズ中に得られた知識に基づいて質問に答えます。

In [ ]:
prompt_data = """あなたは役に立つアシスタントです。質問には簡潔に答えてください。答えに自信がない場合は、「わかりません」と言ってください。

Question: AnyCompany AC8 のパンクしたタイヤを修理するにはどうすればいいですか？
Answer:"""


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** While running the code in upcoming tasks, you might observe *ThrottlingException* messages and retry attempts. Your code includes robust error handling that will automatically retry failed requests with exponential backoff. This is normal behavior when working with service quotas and demonstrates how production-ready applications should handle API limitations.

## タスク3.3: JSON本体を渡してモデルを呼び出し、レスポンスを生成する

In [ ]:
import json
import time
from botocore.exceptions import ClientError
# Model configuration
modelId = "amazon.nova-lite-v1:0"
accept = "application/json"
contentType = "application/json"

# Retry configuration
max_retries = 5
retry_delay = 10  # seconds

def invoke_model_with_retry(bedrock_client, modelId, prompt_text, contentType, accept, max_retries, retry_delay):
    """Invoke the model with retry logic."""
    # Format the prompt for Nova Lite
    body = format_for_nova_lite(prompt_text)
    
    for attempt in range(max_retries):
        try:
            response = bedrock_client.invoke_model(
                modelId=modelId,
                body=json.dumps(body),
                contentType=contentType,
                accept=accept
            )
            
            response_body = json.loads(response.get('body').read())
            return parse_nova_lite_response(response_body)

        except ClientError as error:
            if error.response['Error']['Code'] == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\\n\
                \nTo troubleshoot this issue please refer to the following resources:\\n\
                \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\\n\
                \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
                raise

            elif error.response['Error']['Code'] in ['ThrottlingException', 'ServiceUnavailableException']:
                if attempt < max_retries - 1:
                    print(f"Service capacity reached. Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                    retry_delay *= 2  # Exponential backoff
                else:
                    print("Max retries reached. Unable to invoke the model.")
                    raise

            else:
                print(f"An error occurred: {error}")
                raise

        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            raise

# Main execution
try:
    result = invoke_model_with_retry(bedrock_client, modelId, prompt_data, contentType, accept, max_retries, retry_delay)
    print(result)
except Exception as e:
    print(f"Failed to invoke the model after retries: {e}")




モデルは車のパンクしたタイヤを交換する手順を概説した回答を出しますが、同じ説明はどの車にも当てはまります。残念ながら、これはスペアタイヤのない AnyCompany AC8 には正しい回答ではありません。これは、モデルが車のタイヤ交換手順を含むデータでトレーニングされているためです。

この問題の別の例は、Amazon Tirana など、完全に偽の車のブランドとモデルに同じ質問をしてみることでわかります。

In [ ]:
import json
import time
from botocore.exceptions import ClientError

# Usage
prompt_data = "Amazon Tirana のパンクしたタイヤを修理するにはどうすればいいですか?"

def invoke_model_with_retry(prompt_data, max_retries=5, initial_delay=10):
    modelId = "amazon.nova-lite-v1:0"
    accept = "application/json"
    contentType = "application/json"
    
    # Format the prompt for Nova Lite
    body = json.dumps(format_for_nova_lite(prompt_data))
    
    for attempt in range(max_retries):
        try:
            response = bedrock_client.invoke_model(
                body=body, 
                modelId=modelId, 
                accept=accept, 
                contentType=contentType
            )
            response_body = json.loads(response.get("body").read())
            return parse_nova_lite_response(response_body)

        except ClientError as error:
            error_code = error.response['Error']['Code']
            
            if error_code == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\\n\
                \nTo troubleshoot this issue please refer to the following resources:\\n\
                \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\\n\
                \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
                raise
                
            elif error_code in ['ThrottlingException', 'ServiceUnavailableException', 'ModelStreamLimitExceededException']:
                if attempt < max_retries - 1:
                    delay = initial_delay * (2 ** attempt)  # Exponential backoff
                    print(f"Service capacity reached. Retrying in {delay} seconds...")
                    print(f"Error: {error}")
                    time.sleep(delay)
                    continue
                else:
                    print(f"Max retries ({max_retries}) reached. Last error: {error}")
                    raise
            else:
                print(f"Unhandled error occurred: {error}")
                raise
                
        except Exception as e:
            print(f"Unexpected error: {e}")
            if attempt < max_retries - 1:
                delay = initial_delay * (2 ** attempt)
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                continue
            else:
                raise

try:
    # Invoke model with retry logic
    result = invoke_model_with_retry(prompt_data)
    # Print the raw result
    if result:
        print(result.strip())
    else:
        print("No response generated from the model.")

except Exception as e:
    print(f"Failed to get response after all retries: {e}")


プロンプトの質問では、モデルは現実的な回答を提供できません。

生成 AI モデルが特定の車種に有効な指示に基づいて回答を提供するために、プロンプトの一部として追加の知識ベースを提供することで、モデルの知識をオンザフライで拡張できます。

これを使用してアプリケーションを改善する方法を見てみましょう。

以下は、AnyCompany AC8 のマニュアルからの抜粋であると仮定します (実際に実在するマニュアルではありませんが、そのように扱います)。 このドキュメントは、Nova Lite コンテキスト ウィンドウに完全に収まるほど短く便利です。

```plain
タイヤとタイヤの空気圧:

タイヤは黒いゴムでできており、車のホイールに取り付けられています。タイヤは、運転、コーナリング、ブレーキングに必要なグリップを提供します。考慮すべき 2 つの重要な要素は、タイヤの空気圧とタイヤの摩耗です。これらは、車の性能とハンドリングに影響を与える可能性があります。

推奨タイヤ空気圧の確認場所:

推奨タイヤ空気圧の仕様は、車の運転席側 B ピラーにある空気圧ラベルに記載されています。または、車のマニュアルを参照してこの情報を入手することもできます。推奨タイヤ空気圧は、速度、乗員数、または車内の最大荷重によって異なる場合があります。

タイヤの空気圧の調整:

タイヤ空気圧を確認するときは、タイヤが冷えているときに行うことが重要です。つまり、車を少なくとも 3 時間放置して、タイヤが周囲温度と同じ温度になるようにします。

タイヤの空気圧を調整するには:

車の推奨タイヤ空気圧を確認します。

エアポンプの指示に従って、タイヤを適切な空気圧に調整します。
車両のセンターディスプレイで、「車の状態」アプリを開きます。
「タイヤ空気圧」タブに移動します。
「空気圧の調整」オプションを押して、アクションを確認します。
タイヤ空気圧を調整するには、30 km/h 以上の速度で車を数分間運転します。

注: 場合によっては、タイヤ空気圧に関する警告記号やメッセージを消すために 15 分以上運転する必要があることがあります。警告が消えない場合は、タイヤを冷ましてから上記の手順を繰り返します。

パンク:

運転中にタイヤがパンクした場合は、タイヤモビリティキットを使用して一時的にパンクを塞ぎ、タイヤを再び膨らませることができます。このキットは通常、車両の荷物スペースの裏地の下に保管されています。

タイヤモビリティキットの使用手順:

車両のテールゲートまたはトランクを開きます。
荷物スペースの裏地を持ち上げて、タイヤモビリティキットにアクセスします。
タイヤモビリティキットに付属の説明書に従って、タイヤのパンクを塞ぎます。
キットを使用した後は、必ず元の場所にしっかりと戻してください。
使用済みのシーラントボトルの廃棄と交換については、AnyCompany または適切なサービスにお問い合わせください。

タイヤ モビリティ キットは一時的な解決策であり、最高時速 80 km/h で最大 10 分または 8 km (いずれか早い方) 走行できるように設計されていることに注意してください。パンクしたタイヤはできるだけ早く交換するか、専門家に修理してもらうことをお勧めします。
```

In [ ]:
context = """タイヤとタイヤの空気圧:

タイヤは黒いゴムでできており、車のホイールに取り付けられています。タイヤは、運転、コーナリング、ブレーキングに必要なグリップを提供します。考慮すべき 2 つの重要な要素は、タイヤの空気圧とタイヤの摩耗です。これらは、車の性能とハンドリングに影響を与える可能性があります。

推奨タイヤ空気圧の確認場所:

推奨タイヤ空気圧の仕様は、車の運転席側 B ピラーにある空気圧ラベルに記載されています。または、車のマニュアルを参照してこの情報を入手することもできます。推奨タイヤ空気圧は、速度、乗員数、または車内の最大荷重によって異なる場合があります。

タイヤの空気圧の調整:

タイヤ空気圧を確認するときは、タイヤが冷えているときに行うことが重要です。つまり、車を少なくとも 3 時間放置して、タイヤが周囲温度と同じ温度になるようにします。

タイヤの空気圧を調整するには:

車の推奨タイヤ空気圧を確認します。

エアポンプの指示に従って、タイヤを適切な空気圧に調整します。
車両のセンターディスプレイで、「車の状態」アプリを開きます。
「タイヤ空気圧」タブに移動します。
「空気圧の調整」オプションを押して、アクションを確認します。
タイヤ空気圧を調整するには、30 km/h 以上の速度で車を数分間運転します。

注: 場合によっては、タイヤ空気圧に関する警告記号やメッセージを消すために 15 分以上運転する必要があることがあります。警告が消えない場合は、タイヤを冷ましてから上記の手順を繰り返します。

パンク:

運転中にタイヤがパンクした場合は、タイヤモビリティキットを使用して一時的にパンクを塞ぎ、タイヤを再び膨らませることができます。このキットは通常、車両の荷物スペースの裏地の下に保管されています。

タイヤモビリティキットの使用手順:

車両のテールゲートまたはトランクを開きます。
荷物スペースの裏地を持ち上げて、タイヤモビリティキットにアクセスします。
タイヤモビリティキットに付属の説明書に従って、タイヤのパンクを塞ぎます。
キットを使用した後は、必ず元の場所にしっかりと戻してください。
使用済みのシーラントボトルの廃棄と交換については、AnyCompany または適切なサービスにお問い合わせください。

タイヤ モビリティ キットは一時的な解決策であり、最高時速 80 km/h で最大 10 分または 8 km (いずれか早い方) 走行できるように設計されていることに注意してください。パンクしたタイヤはできるだけ早く交換するか、専門家に修理してもらうことをお勧めします。"""

##### ここで、抜粋全体を質問とともにモデルに渡します。

In [ ]:
question = " AnyCompany AC8 のパンクしたタイヤを修理するにはどうすればいいですか？"
prompt_data = f"""## の間に提供された情報のみに基づいて質問に答え、ステップバイステップのガイドを提供します。
#
{context}
#

Question: {question}
Answer:"""

### タスク 3.4: boto3 経由でモデルを呼び出してレスポンスを生成する

In [ ]:
import json
import time
from botocore.exceptions import ClientError

def invoke_nova_lite_with_retry(prompt_data, max_retries=5, initial_delay=10):
    # Format the prompt for Nova Lite
    body = json.dumps(format_for_nova_lite(prompt_data))
    modelId = "amazon.nova-lite-v1:0"  
    accept = "application/json"
    contentType = "application/json"

    for attempt in range(max_retries):
        try:
            response = bedrock_client.invoke_model(
                body=body, 
                modelId=modelId, 
                accept=accept, 
                contentType=contentType
            )
            response_body = json.loads(response.get("body").read())
            return parse_nova_lite_response(response_body)

        except ClientError as error:
            error_code = error.response['Error']['Code']
            
            if error_code == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\\n\
                \nTo troubleshoot this issue please refer to the following resources:\\n\
                \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\\n\
                \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
                raise
            
            elif error_code in ['ThrottlingException', 'ServiceUnavailableException', 'ModelStreamLimitExceededException']:
                if attempt < max_retries - 1:
                    delay = initial_delay * (2 ** attempt)  # Exponential backoff
                    print(f"Service capacity reached. Retrying in {delay} seconds...")
                    print(f"Error: {error}")
                    time.sleep(delay)
                    continue
                else:
                    print(f"Max retries ({max_retries}) reached. Last error: {error}")
                    raise
            else:
                print(f"Unhandled error occurred: {error}")
                raise
                
        except Exception as e:
            if attempt < max_retries - 1:
                delay = initial_delay * (2 ** attempt)
                print(f"Unexpected error: {e}")
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                continue
            else:
                print(f"Failed after {max_retries} attempts. Last error: {e}")
                raise

# Usage
try:
    answer = invoke_nova_lite_with_retry(prompt_data)
    print(answer.strip())
except Exception as e:
    print(f"Final error: {e}")


モデルがコンテキストを理解して適切な回答を生成するのに時間がかかるため、応答を数秒間待たなければならなくなり、ユーザーエクスペリエンスが低下する可能性があります。

Bedrock は、モデルがトークンを生成するとサービスが出力を生成するストリーミング機能もサポートしています。これを実装する方法の例を次に示します。

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
import json
import time
from botocore.exceptions import ClientError

def invoke_nova_lite_stream_with_retry(prompt_data, max_retries=5, initial_delay=10):
    body = json.dumps(format_for_nova_lite(prompt_data))
    modelId = "amazon.nova-lite-v1:0"
    accept = "application/json"
    contentType = "application/json"
    
    print(f"Using model: {modelId}")

    for attempt in range(max_retries):
        try:
            response = bedrock_client.invoke_model_with_response_stream(
                body=body, 
                modelId=modelId, 
                accept=accept, 
                contentType=contentType
            )
            
            stream = response.get('body')
            output = []
            i = 1
            
            if stream:
                for event in stream:
                    chunk = event.get('chunk')
                    if chunk:
                        chunk_obj = json.loads(chunk.get('bytes').decode())
                        # Extract text from Nova Lite's response format
                        if 'contentBlockDelta' in chunk_obj and 'delta' in chunk_obj['contentBlockDelta']:
                            text = chunk_obj['contentBlockDelta']['delta'].get('text', '')
                            clear_output(wait=True)
                            output.append(text)
                            display_markdown(Markdown(''.join(output)))
                            i += 1
                return ''.join(output)  # Return the complete output
            else:
                raise Exception("No stream data received")

        except ClientError as error:
            error_code = error.response['Error']['Code']
            
            if error_code == 'AccessDeniedException':
                print(f"\x1b[41m{error.response['Error']['Message']}\\n\
                \nTo troubleshoot this issue please refer to the following resources:\\n\
                \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\\n\
                \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
                raise
            
            elif error_code in ['ThrottlingException', 'ServiceUnavailableException', 'ModelStreamLimitExceededException']:
                if attempt < max_retries - 1:
                    delay = initial_delay * (2 ** attempt)  # Exponential backoff
                    print(f"Service capacity reached. Retrying in {delay} seconds...")
                    print(f"Error: {error}")
                    time.sleep(delay)
                    continue
                else:
                    print(f"Max retries ({max_retries}) reached. Last error: {error}")
                    raise
            else:
                print(f"Unhandled error occurred: {error}")
                raise
                
        except Exception as e:
            if attempt < max_retries - 1:
                delay = initial_delay * (2 ** attempt)
                print(f"Unexpected error: {e}")
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
                continue
            else:
                print(f"Failed after {max_retries} attempts. Last error: {e}")
                raise

# Usage
try:
    # You can adjust retry parameters here
    result = invoke_nova_lite_stream_with_retry(
        prompt_data,
        max_retries=5,
        initial_delay=10
    )

except Exception as e:
    print(f"Final error: {e}")


応答には、タイヤの交換方法に関する要約された手順が示されています。

これで、検索拡張生成 (RAG) または拡張プロセスを活用して、提供された特定のコンテキストと情報に合わせて調整された応答を生成する方法を学習しました。

### 試してみましょう
- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、**タスク 4** に進んでください。